In [45]:
import numpy as np
import matplotlib.pyplot as plt

In [38]:
employ = np.genfromtxt('datasets_cleaned/employment/employ.csv', delimiter=',', dtype=['i8','U64','f8','f8','i8'], names=True)
poly = np.genfromtxt('datasets_cleaned/poly/poly_intake.csv', delimiter=',', dtype=['i8','U8','U64','i8','i8','i8'], names=True)
uni = np.genfromtxt('datasets_cleaned/uni/uni_intake.csv', delimiter=',', dtype=['i8','U8','U64','i8','i8','i8'], names=True)

In [44]:
# Matplotlib magic

['Business' 'Business' 'Architecture and Built Environment'
 'Architecture and Built Environment' 'Dentistry' 'Dentistry' 'Education'
 'Education' 'Engineering' 'Engineering'
 'Arts` Humanities and the Social Sciences'
 'Arts` Humanities and the Social Sciences' 'Health Sciences'
 'Health Sciences' 'Information & Digital Technologies'
 'Information & Digital Technologies' 'Law' 'Law' 'Medicine' 'Medicine'
 'Sciences' 'Sciences' 'Business' 'Business'
 'Architecture and Built Environment' 'Architecture and Built Environment'
 'Dentistry' 'Dentistry' 'Education' 'Education' 'Engineering'
 'Engineering' 'Arts` Humanities and the Social Sciences'
 'Arts` Humanities and the Social Sciences' 'Health Sciences'
 'Health Sciences' 'Information & Digital Technologies'
 'Information & Digital Technologies' 'Law' 'Law' 'Medicine' 'Medicine'
 'Sciences' 'Sciences' 'Business' 'Business'
 'Architecture and Built Environment' 'Architecture and Built Environment'
 'Dentistry' 'Dentistry' 'Education' 'Ed